In [9]:
## import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


In [10]:
## checking versions 
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.4.1
Eager mode:  True
Hub version:  0.11.0
GPU is available


## Doanload IMDB Dataset


In [11]:
train_data, test_data = tfds.load(name='imdb_reviews', split=['train', 'test'], batch_size=-1, as_supervised=True)

In [12]:
X_train, y_train = tfds.as_numpy(train_data)
X_test, y_test = tfds.as_numpy(test_data)

## Explore dataset

In [13]:
print(f"Size of Train and Test datasets :- {len(X_train)}, {len(X_test)}")

Size of Train and Test datasets :- 25000, 25000


In [14]:
## sample ones
for index in range(5):
  print(f"{index} sentence: {X_train[index]}\n Label :- {y_train[index]}")

0 sentence: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 Label :- 0
1 sentence: b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film w

## Build the Model


##### Pretrained embedding model 

In [17]:
# embedding_model = 'https://tfhub.dev/google/nnlm-en-dim50/2'   84%
embedding_model = 'https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2'
embedding_layer = hub.KerasLayer(embedding_model, input_shape=[], dtype=tf.string, trainable=True)
sample = embedding_layer(X_train[:2])

## build model
model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

print(f"Summary Of the Model :- {model.summary()}")

model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## create validation dataset
X_val = X_train[:10000]
partial_X_train = X_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

history = model.fit(partial_X_train, partial_y_train, batch_size=512, epochs=50, verbose=1, validation_data=(X_val, y_val))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 124,652,001
Trainable params: 124,652,001
Non-trainable params: 0
_________________________________________________________________
Summary Of the Model :- None
Epoch 1/50
30/30 [==============================] - 3s 94ms/step - loss: 0.6463 - accuracy: 0.5459 - val_loss: 0.4771 - val_accuracy: 0.7488
Epoch 2/50
30/30 [==============================] - 3s 89ms/step - l

30/30 [==============================] - 3s 94ms/step - loss: 7.7588e-05 - accuracy: 1.0000 - val_loss: 0.5943 - val_accuracy: 0.8883
Epoch 24/50
30/30 [==============================] - 3s 88ms/step - loss: 7.0083e-05 - accuracy: 1.0000 - val_loss: 0.6018 - val_accuracy: 0.8884
Epoch 25/50
30/30 [==============================] - 3s 89ms/step - loss: 5.8136e-05 - accuracy: 1.0000 - val_loss: 0.6097 - val_accuracy: 0.8880
Epoch 26/50
30/30 [==============================] - 3s 90ms/step - loss: 5.3817e-05 - accuracy: 1.0000 - val_loss: 0.6173 - val_accuracy: 0.8882
Epoch 27/50
30/30 [==============================] - 3s 89ms/step - loss: 4.4375e-05 - accuracy: 1.0000 - val_loss: 0.6233 - val_accuracy: 0.8881
Epoch 28/50
30/30 [==============================] - 3s 89ms/step - loss: 4.2732e-05 - accuracy: 1.0000 - val_loss: 0.6299 - val_accuracy: 0.8879
Epoch 29/50
30/30 [==============================] - 3s 89ms/step - loss: 4.2008e-05 - accuracy: 1.0000 - val_loss: 0.6361 - val_accurac

## Model evaluating 

In [18]:
results = model.evaluate(X_test, y_test)

782/782 [==============================] - 5s 6ms/step - loss: 0.9012 - accuracy: 0.8603
